In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
listings=pd.read_csv("/content/drive/My Drive/BT4222_Project/final_datasets/listings_final_version.csv")
customers=pd.read_csv("/content/drive/My Drive/BT4222_Project/final_datasets/customers_final_version.csv")

In [4]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222 entries, 0 to 12221
Data columns (total 68 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12222 non-null  int64  
 1   description                     12222 non-null  object 
 2   host_since                      12222 non-null  object 
 3   host_is_superhost               12222 non-null  int64  
 4   latitude                        12222 non-null  float64
 5   longitude                       12222 non-null  float64
 6   accommodates                    12222 non-null  int64  
 7   bathrooms                       12222 non-null  float64
 8   bedrooms                        12222 non-null  float64
 9   beds                            12222 non-null  float64
 10  amenities                       12222 non-null  object 
 11  price                           12222 non-null  float64
 12  minimum_nights                  

In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130602 entries, 0 to 130601
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   reviewer_id                  130602 non-null  int64  
 1   listing_id                   130602 non-null  int64  
 2   date                         130602 non-null  object 
 3   comments                     130602 non-null  object 
 4   price                        130602 non-null  float64
 5   review_scores_rating         130602 non-null  float64
 6   review_scores_accuracy       130602 non-null  float64
 7   review_scores_cleanliness    130602 non-null  float64
 8   review_scores_checkin        130602 non-null  float64
 9   review_scores_communication  130602 non-null  float64
 10  review_scores_location       130602 non-null  float64
 11  review_scores_value          130602 non-null  float64
 12  polarity_score               130602 non-null  float64
 13 

# Knowledge Recommender

In [6]:
import pandas as pd
import numpy as np

In [7]:
df_kr = listings[['host_is_superhost', 'nearby_sites_rating', 'bathrooms', 'bedrooms', 'beds', 'instant_bookable','amenities_list', 'price']]

In [8]:
df_kr.head()

,host_is_superhost,nearby_sites_rating,bathrooms,bedrooms,beds,instant_bookable,amenities_list,price
0,0,1.135049,1.0,1.0,1.0,0,"['TV', 'Cable TV', 'Wifi', 'Air conditioning',...",200.0
1,0,1.288425,1.0,1.0,1.0,0,"['TV', 'Cable TV', 'Internet', 'Wifi', 'Air co...",76.0
2,0,1.982270,1.0,1.0,1.0,0,"['Internet', 'Wifi', 'Air conditioning', 'Pool...",80.0
3,0,2.550789,1.0,0.0,3.0,0,"['TV', 'Internet', 'Wifi', 'Air conditioning',...",143.0
4,0,1.689570,1.0,1.0,1.0,0,"['TV', 'Wifi', 'Air conditioning', 'Kitchen', ...",85.0


In [9]:
def build_chart(df, host_is_superhost, nearby_sites_rating, bathrooms, bedrooms, beds, instant_bookable, *amenities):
    listings = df.copy()
    
    # Filter based on the condition
    listings = listings[(listings['host_is_superhost'] == host_is_superhost) & 
                    (listings['nearby_sites_rating'] >= nearby_sites_rating) & 
                    (listings['bathrooms'] >= bathrooms) & 
                    (listings['bedrooms'] >= bedrooms) & 
                    (listings['beds'] >= beds) &
                    (listings['instant_bookable'] == instant_bookable)]

    for a in amenities:
      listings = listings[listings['amenities_list'].str.contains(a)]

    q_listings = listings.sort_values('price')
    
    return q_listings

In [10]:
build_chart(df_kr, 1, 2, 2, 2, 2, 0, "TV", "Air conditioning", "Kitchen").head()

,host_is_superhost,nearby_sites_rating,bathrooms,bedrooms,beds,instant_bookable,amenities_list,price
2315,1,2.105276,3.0,2.0,2.0,0,"['TV', 'Internet', 'Wifi', 'Air conditioning',...",175.0
7586,1,2.430885,2.0,2.0,2.0,0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",179.0
5600,1,2.334235,2.0,2.0,2.0,0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",188.0
6189,1,2.881733,2.0,2.0,2.0,0,"['TV', 'Cable TV', 'Internet', 'Wifi', 'Air co...",205.0
5416,1,2.396607,2.0,2.0,3.0,0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",246.0


# Content-based Recommendation

In [11]:
df_cbr = listings.copy()

In [12]:
from ast import literal_eval

df_cbr['amenities_list'] = df_cbr.amenities_list.apply(literal_eval)

In [13]:
df_cbr.head(1)

,id,description,host_since,host_is_superhost,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,email,phone,reviews,jumio,offline_government_id,selfie,government_id,identity_manual,work_email,manual_online,manual_offline,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average
0,1101652,Explore one of Singapore's best kept heritage ...,2012-11-15,0,1.29737,103.90068,4,1.0,1.0,1.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",""Wheelc...",200.0,5,800,800.0,0,0,1,1.135049,1.81344,3.174312,4.5,2.0,1,1,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"[TV, Cable TV, Wifi, Air conditioning, Wheelch...",1,1,1,1,0,0,0,0,0,0,0.776522


In [14]:
listings_with_name=pd.read_csv("/content/drive/My Drive/BT4222_Project/datasets/listings_cleaned.csv", low_memory=False)

In [15]:
listings_with_name.head(1)

,id,name,description,neighborhood_overview,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating
0,49091,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,I am a private tutor by profession. My husband...,within a few hours,100%,f,Woodlands,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,Woodlands,North Region,1.44255,103.7958,Apartment,Private room,1,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$81.00,180,360,180,180,360,360,180.0,360.0,58 months ago,30,60,90,365,1,0,94.0,10.0,10.0,10.0,10.0,8.0,8.0,f,2,0.02,0.995731,0.617762,4.295315,4.595264


In [16]:
name_dict = {}
for index, listing in listings_with_name.iterrows():
  name_dict[listing['id']] = str(listing['name']).title()

In [17]:
df_cbr['name'] = df_cbr['id'].apply(lambda x: name_dict[x])

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df_cbr['description'])

tfidf_matrix.shape

(12222, 18231)

In [19]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [20]:
indices = pd.Series(df_cbr.index, index=df_cbr['name']).drop_duplicates()

In [21]:
def content_recommender(name, cosine_sim, df, indices):
    idx = indices[name]

    # Get the pairwsie similarity scores of all listings with that listing
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the listings based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar listings. Ignore the first one.
    sim_scores = sim_scores[1:11]

    # Get the listing indices
    listings_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar listings
    return df['name'].iloc[listings_indices]

In [22]:
content_recommender("Cozy Place In Singapore", cosine_sim, df_cbr, indices)

7570            2Br 2Bath In Katong Near F&B (Min 3 Mths)
2308                    Near To Central With Free Parking
3024     Modern 2Br Apt In Katong, Free Parking, Near F&B
3836     Humble & Conveniently Located 1Br In Katong Area
6257     Humble & Conveniently Located 1Br In Katong Area
2049                                     East Coast Haven
6336     Humble & Conveniently Located 1Br In Katong Area
12220          Kampong Living Cosy Room At Coliving Space
4369           Spacious 3Br Home With East Coast Sea View
10348    Humble & Conveniently Located 1Br In Katong Area
Name: name, dtype: object

# Collaborative Filtering Recommendation

In [23]:
!pip install surprise

In [24]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import KFold

In [25]:
df_cfr = customers.copy()

In [26]:
counts = df_cfr['listing_id'].value_counts()
df_cfr = df_cfr[df_cfr['listing_id'].isin(counts[counts >= 10].index)]

In [27]:
counts1 = df_cfr['reviewer_id'].value_counts()
df_cfr = df_cfr[df_cfr['reviewer_id'].isin(counts1[counts1 > 2].index)]

In [28]:
df = df_cfr[['reviewer_id','listing_id','polarity_score' ]]
df.head()
print(df.nunique())

reviewer_id       1878
listing_id        2049
polarity_score    1819
dtype: int64


In [29]:
reader = Reader(rating_scale=(-1, 1))
data = Dataset.load_from_df(df[['reviewer_id', 'listing_id', 'polarity_score']], reader)

In [30]:
raw_ratings = data.raw_ratings

import random
from surprise.model_selection import GridSearchCV
# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005,0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[100,500]} # tuning
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
print('best RMSE score')
print(grid_search.best_score['rmse'])

Grid Search...
best RMSE score
0.2519522046750071


In [31]:
print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']

combination of parameters that gave the best RMSE score
{'n_epochs': 10, 'lr_all': 0.1, 'reg_all': 0.6, 'n_factors': 500}


In [32]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

from surprise import accuracy
# Compute biased accuracy on A
testset= trainset.build_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))

Biased accuracy on A,   RMSE: 0.1556
len(predictions)
6734


In [33]:
# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)
print('len(predictions)')
print(len(predictions))

Unbiased accuracy on B, RMSE: 0.2430
len(predictions)
749


In [34]:
# define a cross-validation iterator
kf = KFold(n_splits=7)
algo = SVD(n_factors=500, n_epochs=5, lr_all=0.1)

for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


RMSE: 0.3025
RMSE: 0.2643
RMSE: 0.3017
RMSE: 0.3013
RMSE: 0.2961
RMSE: 0.2881
RMSE: 0.3131


In [35]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

testset= trainset.build_anti_testset()
predictionsAll = algo.test(testset)
print('Accuracy on whole data set,', end='   ')
accuracy.rmse(predictionsAll,verbose=True)
print('len(predictions)')
print(len(predictionsAll))

Accuracy on whole data set,   RMSE: 0.1919
len(predictions)
3680594


In [36]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 3 recommendations')

getting top 3 recommendations


In [37]:
from collections import defaultdict
top3_recommendations = get_top3_recommendations(predictionsAll,3)

print('Total predictions calculated are %d' % len(predictionsAll))

Total predictions calculated are 3680594


In [38]:
dfo = pd.DataFrame(columns=['UserId', 'Recommended Listing,Rating'])
i=0;
for uid, user_ratings in top3_recommendations.items():
    #print(uid, top3_recommendations[uid])
    row = [uid, top3_recommendations[uid]]
    dfo.loc[i] = row
    i=i+1
dfo.to_csv('/content/drive/My Drive/BT4222_Project/submission.csv', index = False)

print("wrote Recommendations for each user in csv file")     
        

wrote Recommendations for each user in csv file


# Hybrid Recommendation

In [39]:
#Build the SVD based Collaborative filter
from surprise import SVD, Reader, Dataset
from surprise.model_selection import KFold

reader = Reader(rating_scale=(-1, 1))
ratings_hr = customers.copy() 
data = Dataset.load_from_df(ratings_hr[['reviewer_id', 'listing_id', 'polarity_score']], reader)
kf = KFold(n_splits=5)
kf.split(data)
svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

In [40]:
id_to_name = df_cbr.copy().set_index('id')
name_to_id = df_cbr.copy().set_index('name')
indices = pd.Series(df_cbr.index, index=df_cbr['name']).drop_duplicates()

In [41]:
def hybrid(reviewer_id, name):
    idx = indices[name]
    listing_id = name_to_id.loc[name]['id']
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    listings_indices = [i[0] for i in sim_scores]
    listing = listings.iloc[listings_indices].copy()
    listing['estimate'] = listing['id'].apply(lambda x: svd.predict(reviewer_id, x).est)
    listing = listing.sort_values('estimate', ascending=False)
    return listing.head(5)

In [42]:
hybrid(8557223, "Cozy Place In Singapore")

,id,description,host_since,host_is_superhost,latitude,longitude,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,maximum_nights_avg_ntm,number_of_reviews_ltm,instant_bookable,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating,host_response_time_cross_rate,email,phone,reviews,jumio,offline_government_id,selfie,government_id,identity_manual,work_email,manual_online,manual_offline,x0_Central Region,x0_East Region,x0_North Region,x0_North-East Region,x0_West Region,x0_apartment,x0_bed and breakfast,x0_boat,x0_boutique hotel,x0_bungalow,x0_condominium,x0_guest suite,x0_hostel,x0_hotel,x0_house,x0_loft,x0_other,x0_townhouse,x0_Entire home/apt,x0_Hotel room,x0_Private room,x0_Shared room,amenities_list,Wifi,Air conditioning,Kitchen,Parking,Outdoor,Kids,Pets allowed,Workspace,Gym,Breakfast,availability_average,estimate
9362,18484277,The property is situated in Katong neighbourho...,2016-04-09,1,1.30735,103.89918,4,0.0,1.0,1.0,"[""Dryer"", ""Shampoo"", ""Dedicated workspace"", ""W...",100.0,30,1125,1125.0,1,1,120,1.114351,1.176417,2.847086,4.5,4.00,1,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['Dryer', 'Shampoo', 'Dedicated workspace', 'W...",1,1,1,0,0,0,0,0,1,0,0.191819,0.987482
11377,41722716,The property is situated in Katong neighbourho...,2016-04-09,1,1.30599,103.89951,2,0.0,1.0,1.0,"[""Shampoo"", ""Crib"", ""Long term stays allowed"",...",90.0,30,1125,1125.0,1,0,120,1.119442,1.236447,2.854856,4.5,4.00,1,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['Shampoo', 'Crib', 'Long term stays allowed',...",1,1,1,0,0,0,0,0,1,0,1.000000,0.927539
4307,14387136,We want all our guests to enjoy Singapore! Cos...,2014-06-29,1,1.30996,103.90159,6,1.0,2.0,3.0,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",184.0,28,1125,1125.0,25,0,5,1.090599,1.107006,2.522305,4.5,4.00,1,1,1,1,1,1,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['TV', 'Internet', 'Wifi', 'Air conditioning',...",1,1,1,1,1,0,0,1,0,0,0.713661,0.891504
6336,19780831,The property is situated in Katong neighbourho...,2016-04-09,1,1.30554,103.90064,2,1.0,1.0,1.0,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,""Pets...",190.0,30,1125,1125.0,2,1,335,1.116663,1.370063,2.752297,4.5,4.00,1,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['TV', 'Wifi', 'Air conditioning', 'Pool', 'Ki...",1,1,1,0,0,0,1,1,1,0,0.000000,0.831819
3382,7627981,"This huge apartment on the High floor, unblock...",2014-05-15,0,1.31321,103.89285,4,2.0,2.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",158.0,5,1124,1124.0,0,0,9,1.113572,0.475441,3.455570,4.5,2.67,1,1,1,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"['TV', 'Internet', 'Wifi', 'Air conditioning',...",1,1,1,1,0,0,0,1,0,0,0.353272,0.802610


# Hybrid Recommendation 2.0

In [43]:
customers.head()

,reviewer_id,listing_id,date,comments,price,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,polarity_score,sentiment
0,8557223,49091,2013-10-21,Fran was absolutely gracious and welcoming. Ma...,81.0,94.0,10.0,10.0,10.0,10.0,8.0,8.0,0.9615,pos
1,1356099,50646,2014-04-18,A comfortable room in a smart condo developmen...,80.0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,0.9484,pos
2,15222393,50646,2014-06-05,Stayed over at Sujatha's house for 3 good nigh...,80.0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,0.9879,pos
3,5543172,50646,2014-07-02,It's been a lovely stay at Sujatha's. The room...,80.0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,0.9627,pos
4,817532,50646,2014-07-08,"We had a great experience. A nice place, an am...",80.0,91.0,9.0,10.0,10.0,10.0,9.0,9.0,0.9274,pos


In [44]:
df_hr = customers.copy()
df_hr['overall_review'] = (df_hr.review_scores_rating/100 + (df_hr.review_scores_accuracy + df_hr.review_scores_cleanliness + df_hr.review_scores_checkin + \
df_hr.review_scores_communication + df_hr.review_scores_location + df_hr.review_scores_value)/10)/7
df_hr = df_hr.drop(columns = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value'])

In [45]:
df_hr.head()

,reviewer_id,listing_id,date,comments,price,polarity_score,sentiment,overall_review
0,8557223,49091,2013-10-21,Fran was absolutely gracious and welcoming. Ma...,81.0,0.9615,pos,0.934286
1,1356099,50646,2014-04-18,A comfortable room in a smart condo developmen...,80.0,0.9484,pos,0.944286
2,15222393,50646,2014-06-05,Stayed over at Sujatha's house for 3 good nigh...,80.0,0.9879,pos,0.944286
3,5543172,50646,2014-07-02,It's been a lovely stay at Sujatha's. The room...,80.0,0.9627,pos,0.944286
4,817532,50646,2014-07-08,"We had a great experience. A nice place, an am...",80.0,0.9274,pos,0.944286


In [46]:
df_hr['preference'] = (df_hr['polarity_score']+1)*df_hr['overall_review']/2
df_hr = df_hr.drop(columns = ['date','comments','polarity_score','overall_review','sentiment'])

In [47]:
df_hr.describe()

,reviewer_id,listing_id,price,preference
count,1.306020e+05,1.306020e+05,130602.000000,130602.000000
mean,9.486763e+07,1.719744e+07,133.235578,0.827837
std,8.335650e+07,1.036867e+07,134.407248,0.160834
min,2.770000e+02,4.909100e+04,0.000000,0.001640
25%,2.741515e+07,8.364495e+06,62.000000,0.776743
50%,6.787791e+07,1.650210e+07,100.000000,0.875138
75%,1.452399e+08,2.400210e+07,169.000000,0.933538
max,3.850473e+08,4.729685e+07,9999.000000,0.999850


In [48]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_hr[['reviewer_id', 'listing_id', 'preference']], reader)
data_hr = data

In [49]:
raw_ratings = data.raw_ratings

import random
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(0.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

In [ ]:
# Select your best algo with grid search.
print('Grid Search...')

param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005, 0.1],
              'reg_all': [0.4, 0.6], 'n_factors':[100,200,500]} # tuning
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)
print('best RMSE score')
print(grid_search.best_score['rmse'])

Grid Search...


In [ ]:
print('combination of parameters that gave the best RMSE score')
print(grid_search.best_params['rmse'])
algo = grid_search.best_estimator['rmse']

In [ ]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
testset= trainset.build_testset()
predictions = algo.test(testset)
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions,verbose=True)
print('len(predictions)')
print(len(predictions))

In [ ]:
# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)
print('len(predictions)')
print(len(predictions))

In [ ]:
# define a cross-validation iterator
kf = KFold(n_splits=7)
algo = SVD(n_factors=100, n_epochs=5, lr_all=0.1) # update

for trainset, testset in kf.split(data_hr):
    # train and test algorithm.
    algo.fit(trainset)
    
    predictions = algo.test(testset)
    
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

testset= trainset.build_anti_testset()
predictionsAll = algo.test(testset)
print('Accuracy on whole data set,', end='   ')
accuracy.rmse(predictionsAll,verbose=True)
print('len(predictions)')
print(len(predictionsAll))

In [ ]:
def get_top3_recommendations(predictions, topN = 3):    
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]    
    return top_recs
print('getting top 3 recommendations')

In [ ]:
top3_recommendations = get_top3_recommendations(predictionsAll,3)

In [ ]:
dfo = pd.DataFrame(columns=['UserId', 'Recommended Listing, Rating'])
i=0;
for uid, user_ratings in top3_recommendations.items():
    row = [uid, top3_recommendations[uid]]
    dfo.loc[i] = row
    i=i+1
dfo.to_csv('/content/drive/My Drive/BT4222_Project/svd_prediction_on_preference).csv', index = False)